In [246]:

import mysql.connector
import pandas as pd
conn = mysql.connector.connect(
    host = "database-1.cd2is2gsweff.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "admin",
    password = "kim82458529",
    database = "amr_base"
)
cursor = conn.cursor(buffered=True)
cursor.execute("SELECT * FROM yousinsa")
columns = [i[0] for i in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns=columns)
cursor.close()
conn.close()



In [247]:
import ast
import pandas as pd
def get_score(df, user_age, user_sex, middle_category):
    # 좋아요, 리뷰 만족도
    like_list = []
    for idx, data in df.iterrows():
        try:
            like_list.append(data["likes"] / data["review"])
        except ZeroDivisionError:
            like_list.append(0)
    likes_list = [like / max(like_list) for like in like_list]
    df['만족도'] = likes_list
    # 판매량
    df["total"] = df["sold"] * df["price_member"]
    df["total"] = df["total"] / max(df["total"])
    # 선호나이
    pd.set_option("mode.chained_assignment", None)
    for a in range(len(df["age"])):
        try:
            df["age"][a] = list(map(int, df["age"][a].split("~")))
            df["age"][a] = int((df["age"][a][0] + df["age"][a][1])/2)
        except:
            if df["age"][a] != [0]:
                df["age"][a] = df["age"][a][0]
            else:
                df["age"][a] = 0
            
    df["age"] = abs(df["age"] - user_age)
    df["age"] = df["age"] / max(df["age"])
    # 브랜드별 만족도를 최대 만족도로 나누어 정규화
    max_brand_satisfaction = df.groupby('brand')['만족도'].max()
    df['brand 만족도'] = df.apply(lambda row: row['만족도'] / max_brand_satisfaction.get(row['brand'], 1), axis=1)
    # sex -> 리스트화 후 input과 다르면 0
    # 총점 계산
    df["총점"] = df["만족도"] * 4 + df["age"] * 1 + df["total"] * 3 + df["brand 만족도"] * 2
    # 사용자 input 값에 따른 선별
    df.loc[~df['sex'].apply(lambda x: user_sex in x), '총점'] = 0
    df.loc[df["sort1"] != middle_category, '총점'] = 0
    new_df = df.copy()  # 원본 DataFrame을 변경하지 않도록 복사본 사용
    return new_df
# 예시 호출
result_df = get_score(df, 20, "남성", "후드 집업")
result_df

/tmp/ipykernel_23759/2221084474.py:23: RuntimeWarning: invalid value encountered in scalar divide
  df['brand 만족도'] = df.apply(lambda row: row['만족도'] / max_brand_satisfaction.get(row['brand'], 1), axis=1)


,ranking,link,name,brand,sex,sale,sort,sort1,likes,review,price_member,price_nomember,tag,number,sold,age,만족도,total,brand 만족도,총점
0,0,https://www.musinsa.com/app/goods/1778404?loc=...,2WAY 스웻 후드 집업 (MELANGE GREY),toffee,"['남성', '여성']",13,아우터,후드 집업,218205,81445,34710,45000,"후드, 집업후드, 트레이닝세트, 셋업, 세트, 후드티, 오버핏후드티, 무지, 그레이...",1778404,99541,NaN,0.004311,0.694426,0.061007,NaN
1,1,https://www.musinsa.com/app/goods/3777826?loc=...,LONELY/LOVELY WASHED HOODIE ZIP-UP_GRAY,nohant,['여성'],0,아우터,후드 집업,56,0,186120,188000,"후드집업, 후드\집업, 후드집업추천, 오버핏, 집업후드, 론리러블리",3777826,0,NaN,0.000000,0.000000,0.000000,0.0
2,2,https://www.musinsa.com/app/goods/3493145?loc=...,[SET] Very Beloa Hood Zip-up & Very Beloa Pant...,avan,['여성'],0,아우터,후드 집업,5040,428,80899,130000,"후드집업, 후드, 집업후드, 스웻셔츠, 후디, 집업, 패키지",3493145,1335,NaN,0.018947,0.021707,0.075809,0.0
3,3,https://www.musinsa.com/app/goods/2897038?loc=...,폴라베어 와펜 후드집업 (JM5TZU764IV),jeep,['남성'],0,아우터,후드 집업,1689,36,69500,139000,"아울렛, 후드티, 오버핏후드티",2897038,242,NaN,0.075489,0.003380,0.226651,NaN
4,4,https://www.musinsa.com/app/goods/2730700?loc=...,[SET] 크롭후디 OR 하프집업 셋업_멜란지,partimentowomen,['여성'],25,아우터,후드 집업,17096,905,57874,86900,"스웻셔츠, 오버핏, 맨투맨, 스웨트셔츠, 집업, 후디, 후드집업",2730700,1193,NaN,0.030395,0.013877,0.057593,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5672,85,https://www.musinsa.com/app/goods/320924?loc=g...,소가죽 첼시부츠 로빈 브라운,daytrip,['남성'],0,신발,부츠,16,1,88021,89000,"신발, 워커, 첼시부츠",320924,0,NaN,0.025744,0.000000,1.000000,0.0
5673,86,https://www.musinsa.com/app/goods/392776?loc=g...,4.7cm 카스타솔 미니멀 지퍼 앵클부츠 3335 [BLACK FG],firenzeatelier,['남성'],10,신발,부츠,176,15,164020,205000,"앵글부츠, 블랙, 가죽부츠, 수제화부츠, 지퍼부츠, 앵클부츠",392776,0,NaN,0.018879,0.000000,0.278261,0.0
5674,87,https://www.musinsa.com/app/goods/426885?loc=g...,[비브람] 군용워커 모티브 풀그레인 지퍼디테일 워커 9300 [BLACK FG],firenzeatelier,['남성'],0,신발,부츠,339,10,208826,261000,"군용워커, 군화, 레이스업부츠, 블랙, 남자부츠, 수제화, 비브람",426885,0,NaN,0.054545,0.000000,0.803953,0.0
5675,88,https://www.musinsa.com/app/goods/451852?loc=g...,여성 팀버랜드 6인치 프리미엄 부츠 / 10361-713,timberland,['여성'],0,신발,부츠,6084,414,265052,268000,"부츠, 6인치, 6인치프리미엄, 팀버랜드우먼, 팀버랜드부츠, 6인치부츠, 시그니처,...",451852,263,NaN,0.023645,0.014011,0.361962,0.0


In [241]:
user_age = 20 #########유저 나이 들어갈곳
pd.set_option("mode.chained_assignment", None)
for a in range(len(df["age"])):
    try:
        df["age"][a] = list(map(int, df["age"][a].split("~")))
        df["age"][a] = int((df["age"][a][0] + df["age"][a][1])/2)
    except:
        if df["age"][a] != [0]:
            df["age"][a] = df["age"][a][0]
        else:
            df["age"][a] = 0
        
df["age"] = abs(df["age"] - user_age)
df["age"] = df["age"] / max(df["age"])



In [136]:
df["total"] = df["sold"] * df["price_nomember"]
df["total"] = df["total"] / max(df["total"])
(df["total"])


0       0.529478
1       0.000000
2       0.020514
3       0.003976
4       0.012254
          ...   
5672    0.000000
5673    0.000000
5674    0.000000
5675    0.008332
5676    0.002313
Name: total, Length: 5677, dtype: float64

0       0.525
1         0.0
2       0.525
3        0.45
4        0.65
        ...  
5672      0.0
5673      0.0
5674      0.0
5675    0.525
5676     0.65
Name: age, Length: 5677, dtype: object

In [165]:
import ast
cate = "후드 집업"
df["가격"] = df["price_member"]
df["sex"] = df.apply(lambda x: ast.literal_eval(x.sex), axis=1)
df.loc[df['sex'].apply(lambda x: '남성' in x), 'price_member'] = 0
#df["남성" in df["sort1"]].가격


In [220]:
#df["sex"].isin(["남성","여성"])
df["sex"]

0       [남성, 여성]
1           [여성]
2           [여성]
3           [남성]
4           [여성]
          ...   
5672        [남성]
5673        [남성]
5674        [남성]
5675        [여성]
5676        [남성]
Name: sex, Length: 5677, dtype: object

,ranking,link,name,brand,sex,sale,sort,sort1,likes,review,price_member,price_nomember,tag,number,sold,age,total,가격


In [233]:
df.loc[~df['sex'].apply(lambda x: '남성' in x), 'price_member'] = 0
df

SyntaxError: invalid syntax (2737650753.py, line 1)

In [234]:
df["sort1"]

0       후드 집업
1       후드 집업
2       후드 집업
3       후드 집업
4       후드 집업
        ...  
5672       부츠
5673       부츠
5674       부츠
5675       부츠
5676       부츠
Name: sort1, Length: 5677, dtype: object